# Example of a Gibson Assembly in pydna
> Visit the full library documentation [here](https://bjornfjohansson.github.io/pydna/)

This example showcases a workflow of modelling Gibson assembly to clone gene fragments into plasmids for synthetic biology. The biological example is sourced [here](https://www.nature.com/articles/nmeth.1318#MOESM319), from the original Gibson assembly paper. This example constructs a synthetic pCC1BAC plasmid by joining sequence fragments from Ruminiclostridium (Clostridium) cellulolyticum. The R. cellulolyticum fragments joined are termed F1, F2, and F3, as in the paper.

Source files can be found alongside this notebook, if you would like to follow along. Annotations are made alongside the code to describe key steps.


In [ ]:
# Importing all necessary classes and methods

from pydna.parsers import parse
from pydna.tm import tm_default
from pydna.amplify import pcr
from pydna.dseqrecord import Dseqrecord
from pydna.assembly import Assembly

In [ ]:
# Parsing the files

pCC1BAC_path = "./pCC1BAC.gb"
gene_path = "./R_cellulolyticum.fasta"
pCC1BAC_docs = parse(pCC1BAC_path)
gene_docs = parse(gene_path)

# Printing the parsed files

print(pCC1BAC_docs[0].format("gb"))
# print(gene_docs[0].format("fasta"))

LOCUS       EU140750                8128 bp    DNA     circular SYN 15-SEP-2007
DEFINITION  Cloning vector pCC1BAC, complete sequence.
ACCESSION   EU140750
VERSION     EU140750.1
KEYWORDS    .
SOURCE      Cloning vector pCC1BAC
  ORGANISM  Cloning vector pCC1BAC
            other sequences; artificial sequences; vectors.
REFERENCE   1  (bases 1 to 8128)
  AUTHORS   EPICENTRE Biotechnologies.
  TITLE     Direct Submission
  JOURNAL   Submitted (23-AUG-2007) 726 Post Road, Madison, WI 53713, USA
FEATURES             Location/Qualifiers
     source          1..8128
                     /organism="Cloning vector pCC1BAC"
                     /mol_type="other DNA"
                     /db_xref="taxon:468515"
ORIGIN
        1 gcggccgcaa ggggttcgcg tcagcgggtg ttggcgggtg tcggggctgg cttaactatg
       61 cggcatcaga gcagattgta ctgagagtgc accatatgcg gtgtgaaata ccacacagat
      121 gcgtaaggag aaaataccgc atcaggcgcc attcgccatt cagctgcgca actgttggga
      181 agggcgatcg gtgcgggcct cttcgctatt acgccagct

In [ ]:
# Importing pre-designed primers for the PylRS insert fragment. 

F1_For = "GCAGCTTCAAGTCCTGCAAACAAGGTGTACCAGGATCGTT" #Remember that all sequnces are written from 5'-3'
F1_Rev = "GATTTCAGTGTAGTTAGGGCCAGTTGAATTCAAACCTGCC"
F2_For = "GGCAGGTTTGAATTCAACTGGCCCTAACTACACTGAAATC"
F2_Rev = "CTTGGTGCCATCAGCATTGTTCTCTGTACCGCCCACTGTC"
F3_For = "GACAGTGGGCGGTACAGAGAACAATGCTGATGGCACCAAG"
F3_Rev = "CAGTTGAATAATCATGTGTTCCTGCGGCAAATGCAGTACC"
BACF1_For = "AACGATCCTGGTACACCTTGTTTGCAGGACTTGAAGCTGCgcggccgcgatcctctagagtcgacctg"
BACF3_Rev = "GGTACTGCATTTGCCGCAGGAACACATGATTATTCAACTGgcggccgccgggtaccgagctcgaattc"

In [ ]:
# (Optional) Checking that the primer Tm are matching

print(tm_default(F1_For)) # F1_For and F1_Rev have been used as examples here
print(tm_default(F1_Rev))

76.10205760701717
72.57935286927591


In [ ]:
# Checking primer specificity using the pcr function. An error message is returned if otherwise.

pcr_product_F1 = pcr(F1_For, F1_Rev, gene_docs[0], limit=40)
pcr_product_F2 = pcr(F2_For, F2_Rev, gene_docs[0], limit=40)
pcr_product_F3 = pcr(F3_For, F3_Rev, gene_docs[0], limit=40)
pcr_product_BAC = pcr(BACF1_For, BACF3_Rev, pCC1BAC_docs[0], limit=69)

# Printing out the PCR results

print(pcr_product_F1.format("gb"))
print(pcr_product_F2.format("gb"))
print(pcr_product_F3.format("gb"))
print(pcr_product_BAC.format("fasta"))


ValueError: No PCR product! Template EU140750 8128 bp circular limit=69:
No forward primers anneal...
No reverse primers anneal...

In [ ]:
# Performing the Gibson Assembly. Note that the assembly class parameters should be given as a list.

assembled = Assembly([Dseqrecord(pcr_product_F1), Dseqrecord(pcr_product_F2), Dseqrecord(pcr_product_F3), Dseqrecord(pcr_product_BAC)])
assembled_circ = assembled.assemble_circular()

# Printing out the Gibson Assembly product
print(assembled_circ[0])

IndexError: list index out of range